<a href="https://colab.research.google.com/github/TheOtherRealm/DeepLabCut/blob/master/Demo2_OpenSimIKPipeline/Demo%202%20-%20Analyzing%20movement%20data%20with%20OpenSim%20scripting%20in%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Analyzing movement data with OpenSim scripting in Python


## 2.1. Objectives

**Purpose**

The purpose of this tutorial is to demonstrate how [OpenSim](https://opensim.stanford.edu/) [[1]](https://doi.org/10.1109/TBME.2007.901024) can be used to preview motion capture data, and compute joint angles using inverse kinematics (IK), and use OpenSim's analysis tools to compute a quantity of interest from the IK solution.

To diagnose movement disorders and study human movement, biomechanists frequently ask human subjects to perform movements in a motion capture laboratory and use computational tools to analyze these movements. A common step in analyzing a movement is to compute the joint angles of the subject during movement. OpenSim has tools for computing these quantities:

*   **Inverse Kinematics (IK)** is used to compute joint angles.
*   **Analyze Tool** is used to compute a quantity of interest from an OpenSim model.


In this tutorial, you will:

*   Become familiar with OpenSim's tools to visualize models, motions, and data.
*   Solve an inverse kinematics problem using an OpenSim model and experimental marker data collected from OpenCap.
*   Plot a quantity of interest based on the inverse kinematics solution.

**Format**

Each section of the tutorial guides you through certain tools within OpenSim and asks you to answer a few questions. The questions can be answered based on information from OpenSim and basic knowledge of the human musculoskeletal system. As you complete each section of the tutorial, feel free to explore OpenSim and the provided model further on your own.

## 2.2. Set up conda environment and install OpenSim

First, set up the environment by executing the following cell. In the following cell, you will use [`condacolab`](https://github.com/conda-incubator/condacolab) to install [`Miniconda`](https://docs.anaconda.com/miniconda/).

In [7]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


Don't worry if after executing the previous cell you get an error saying that your session has failed. The reason for this is that `condacolab` needs to restart the session for the changes to take effect. Because of this, you have to execute the previous cell first, before executing any other cell.

Now, let's install the OpenSim conda package.

In [8]:
!conda install opensim-org::opensim

Channels:
 - conda-forge
 - opensim-org
Platform: linux-64
Solving environment: | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



## 2.3. Preview the motion capture data

To visualize the motion capture data we collected from OpenCap, we will use the new OpenSim Viewer. Excecute the following lines to install the OpenSim Viewer and its dependencies.

In [9]:
!pip install pygltflib
!conda install -c conda-forge vtk

# Mount OpenSim Viewer code base.
!git clone https://github.com/opensim-org/opensim-viewer-backend.git

# Import the viewport module.
import sys
sys.path.append('opensim-viewer-backend')
import osimViewport

Channels:
 - conda-forge
 - opensim-org
Platform: linux-64
Solving environment: - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

fatal: destination path 'opensim-viewer-backend' already exists and is not an empty directory.


The visualization works by converting OpenSim data and model files into glTF ([Graphics Library Transmission Format](https://en.wikipedia.org/wiki/GlTF)) files. These files are used by the OpenSim Viewer and are universally viewable by other publicly available tools (e.g. Blender, 3D Viewer on Windows, and many other third party tools).

**Download files**

First, let's download all the files needed for this demonstration. Run the following cell to generate a button labeled "Download Files". Click the button to download and extract the files into the working directory.  

In [10]:
# @title Download the mocap data file, model file and setup files { display-mode: "form" }
# @markdown After executing this cell, a download button will appear. Click the button to download and extract the zip file containing the "Squats.trc" mocap file.

import ipywidgets as widgets
from IPython.display import display
import requests
import zipfile
import io

# Define the function to download and extract the zip file
def download_and_extract_zip(url):
    try:
        # Get the content of the URL
        response = requests.get(url)
        response.raise_for_status()

        # Create a ZipFile object from the downloaded content
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            # Extract all the contents into the working directory
            zip_ref.extractall()
            print("Extraction complete!")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download the file: {e}")
    except zipfile.BadZipFile:
        print("The file is not a valid zip file.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Create a button to trigger the download and extraction
download_button = widgets.Button(
    description='Download Files',
    disabled=False,
    button_style='',
    tooltip='Click to download the files needed by the demo',
    icon='download'
)

# Define the button click event handler
def on_button_click(b):
    url = 'https://drive.google.com/uc?export=download&id=1Yqs_xX9MNIIp_QafzF25DEZGOR9e1TRK'
    if url:
        download_and_extract_zip(url)
    else:
        print("Please enter a valid URL.")

# Attach the event handler to the button
download_button.on_click(on_button_click)

# Display the widgets
display(download_button)

Button(description='Download Files', icon='download', style=ButtonStyle(), tooltip='Click to download the file…

To preview the data, we'll pull in a python module to visualize OpenSim supported files.

In [12]:
# Generate a glTF file from the marker data.
ovp = osimViewport.osimViewport()
ovp.addDataFile('Squats.trc')
ovp.show()

NotADirectoryError: [Errno Unable to find file ] /content/Squats.trc

Download the generated file locally, then drag and drop into the viewer below.


In [ ]:
# @title Download the generated GLTF file to your computer { display-mode: "form" }
# @markdown After executing this cell, a button will appear to download the generated GLTF file to your computer.

import ipywidgets as widgets
from IPython.display import display
from google.colab import files
layout = widgets.Layout(width='250px')
button = widgets.Button(description="Download the GLTF file!", layout=layout)
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    files.download('Squats.gltf')

button.on_click(on_button_clicked)
display(button, output)

In [ ]:
# @title Execute this cell to include the OpenSim Viewer { display-mode: "form" }
# @markdown Open the downloaded GLTF file from the Opensim Viewer to visualize.

from IPython.display import IFrame

IFrame('https://dev.d7iaa9a9lxtm6.amplifyapp.com/?css=gui', width=700, height=400)

## 2.4. Run an inverse kinematics problem

Kinematics is the study of motion without considering the forces and moments that produce that motion [[4]](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470549148). The purpose of inverse kinematics (IK) is to estimate the joint angles of a particular subject from experimental data. In this section, you will estimate a subject's joint angles during walking by performing an IK analysis using the subject scaled model and experimentally collected walking data.

For each time step of recorded motion data, IK computes a set of joint angles that put the model in a configuration that "best matches" the experimental kinematics. OpenSim determines this "best match" by solving a weighted least squares optimization problem with the goal of minimizing marker error. Marker error is defined as the distance between an experimental marker and the corresponding model marker. Each marker has an associated weighting value, specifying how strongly that marker's error term should be minimized in the least squares problem. For each time step, the inverse kinematics tool solves for a vector of generalized coordinates (e.g., joint angles), $q$, that minimizes the weighted sum of marker errors, which is expressed as:

$\underset{q}{\text{min}}\Bigg[ \sum_{i \in \text{markers}}{w_i || x_i^{\text{exp}} - x_i(q) ||^2}^{}\Bigg] $

where $q$ is the vector of generalized coordinates (e.g., joint angles), $x_i^{\text{exp}}$ is the position of the experimental marker $i$, $x_i(q)$ is the position of the corresponding model marker $i$ (which depends on $q$), and $w_i$ is the weight associated with marker $i$.

In this example we will use OpenSim's [`InverseKinematicsTool`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1InverseKinematicsTool.html) to solve the inverse kinematics problem. To learn more, visit the [Getting Started with Inverse Kinematics](https://opensimconfluence.atlassian.net/wiki/spaces/OpenSim/pages/53090032/Getting+Started+with+Inverse+Kinematics) page on OpenSim's Confluence documentation site.

**Configure an *InverseKinematicsTool***

The IK Tool helps us solve the Inverse Kinematics problem. We can pass the configuration file we downloaded earlier as a parameter (`ik_setup.xml`).

In [ ]:
# Create an IK object using the configuration file.
import opensim as osim
inverse_kinematics_tool = osim.InverseKinematicsTool('ik_setup.xml')

# Print some information of the config file to check that everything is correct.
print('Inverse Kinematics Tool settings')
print(f'Model File: {inverse_kinematics_tool.get_model_file()}')
print(f'Marker File: {inverse_kinematics_tool.get_marker_file()}')
print(f'Accuracy: {inverse_kinematics_tool.get_accuracy()}')
print(f'Time Range: [{inverse_kinematics_tool.get_time_range(0)}, '
      f'{inverse_kinematics_tool.get_time_range(1)}] s')


# Print weights information
print('Weights: ')
task_set = inverse_kinematics_tool.get_IKTaskSet()
for i in range(task_set.getSize()):
    task = task_set.get(i)
    print(f'{task.getName()}: {task.getWeight()}')

**Perform inverse kinematics**

Finally, we run the IK tool of OpenSim. The results are the motion file (`inverse_kinematics.sto`) and a report of the marker errors during the solution (`_ik_marker_errors.sto`).

In [ ]:
# Run the IK tool.
inverse_kinematics_tool.run()

**Plot marker errors and visualize the motion**

The output file `_ik_marker_errors.sto` provides the markers errors associated with each frame of the generated motion. You can parse this file using a `TimeSeriesTable` and visualize the error obtained per frame.

In [ ]:
# Load the marker errors into a TimeSeriesTable and print the column labels.
table = osim.TimeSeriesTable('_ik_marker_errors.sto')
print(table.getColumnLabels())

We will plot the total squared error, the RMS error for the markers, and the maximum error for the markers. In the next cell you will extract this information from the motion file.


In [ ]:
# Get columns we want to plot, and the independent column (i.e., time).
marker_error_RMS = table.getDependentColumn('marker_error_RMS')
marker_error_max = table.getDependentColumn('marker_error_max')
time = table.getIndependentColumn()

Now, we can plot the marker errors.

In [ ]:
import matplotlib.pyplot as plt

# Plot the marker errors.
fig, axs = plt.subplots(1, 1, figsize=(7, 5))
axs.plot(time, 100*marker_error_RMS.to_numpy(), label='RMS marker error')
axs.plot(time, 100*marker_error_max.to_numpy(), label='Max marker error')
axs.set_xlim([time[0], time[-1]])
axs.set_ylim([0, 10])
axs.set_xlabel('time (s)')
axs.set_ylabel('error (cm)')
axs.set_title('Marker Errors')
axs.grid()
axs.legend()

plt.show()

Use the following exercises to further explore the inverse kinematic solution.

**<u>Exercise 1.</u>  Modify the [`InverseKinematicsTool`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1InverseKinematicsTool.html) we configured above to enable the property `report_marker_locations`. Solve the inverse kinematics problem again.**

**<u>Exercise 2.</u>  Plot the experimental marker trajectories with the model marker trajectories based on new the output from Exercise 1 (`_ik_model_marker_locations.sto`). Which markers have the largest error?**

## 2.5. Visualize the inverse kinematics solution

Next, we will visualize the model and the associated motion computed by inverse kinematics using the OpenSim Viewer.

In [ ]:
# Create a glTF file from the model and IK solution.
ovp = osimViewport.osimViewport()
ovp.addModelAndMotionFiles('LaiUhlrich2022_scaled.osim',
                           ['inverse_kinematics.sto'])
ovp.show()

In [ ]:
# @title Download the generated GLTF file to your computer { display-mode: "form" }
# @markdown After executing this cell, a button will appear to download the generated GLTF file to your computer.

import ipywidgets as widgets
from IPython.display import display
from google.colab import files
layout = widgets.Layout(width='250px')
button = widgets.Button(description="Download the GLTF file!", layout=layout)
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    files.download('LaiUhlrich2022_scaled.gltf')

button.on_click(on_button_clicked)
display(button, output)

In [ ]:
# @title Execute this cell to include the OpenSim Viewer { display-mode: "form" }
# @markdown Open the downloaded GLTF file from the Opensim Viewer to visualize.

from IPython.display import IFrame

IFrame('https://dev.d7iaa9a9lxtm6.amplifyapp.com/?css=gui', width=700, height=400)

## 2.6. Analyze the inverse kinematics solution

OpenSim provides tools to analyze a motion and compute quantities of interest. This is typically done by using the OpenSim [`AnalyzeTool`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1AnalyzeTool.html) to perform analyses on a motion trajectory. Analyses that can be performed with the `AnalyzeTool` include [`BodyKinematics`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1BodyKinematics.html), [`PointKinematics`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1PointKinematics.html), and [`MuscleAnalysis`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1MuscleAnalysis.html). In this example we will use `BodyKinematics` to compute and plot the position of the model's center of center of mass.

A detailed explanation of the `AnalyzeTool` can be found on the [How to Use the Analysis Tool](https://opensimconfluence.atlassian.net/wiki/spaces/OpenSim/pages/53088479/How+to+Use+the+Analysis+Tool) page on Confluence.

**Create an *AnalyzeTool***

The `AnalyzeTool` helps us perform analyses to compute quantities relevant for research questions or clinical assessment. We can pass a configuration file as a parameter. Here, we use the downloaded file `bodykin_setup.xml`.

In [ ]:
# Create an AnalyzeTool object using the configuration file.
analyze_tool = osim.AnalyzeTool('bodykin_setup.xml')

# Print some information of the config file to check that everything is correct.
print(f'Name: {analyze_tool.getName()}')
print(f'Model file: {analyze_tool.getModelFilename()}')
print(f'Time Range: [{analyze_tool.getInitialTime()}, {analyze_tool.getFinalTime()}] s')
print(f'Low-pass filter cutoff frequency: {analyze_tool.getLowpassCutoffFrequency()} Hz')

**Run the *AnalyzeTool***

Finally, we can run analyses by executing the run method of the `AnalyzeTool`.

In [ ]:
# Run the AnalyzeTool.
analyze_tool.run()

**Plot the results**

After running the `AnalyzeTool`, a set of files named `*_BodyKinematics_{acc, vel, pos}_global.sto` are generated. These files contain the results of the tool execution, and you can parse them using a [`TimeSeriesTable`](https://simtk.org/api_docs/opensim/api_docs451/classOpenSim_1_1TimeSeriesTable__.html) to plot the results. The contents are the accelerations, velocities and positions of all bodies as well as the center of mass.

In [ ]:
# Use a TimeSeriesTable to load the results file.
tableBodyKin = osim.TimeSeriesTable(
    'LaiUhlrich2022_scaled_BodyKinematics_pos_global.sto')
print(tableBodyKin.getColumnLabels())

Let's plot the model's center of mass y-position. First, we need to extract the relevant columns from the table.

In [ ]:
# Get the center of mass position and the time vector.
com_y_pos = tableBodyKin.getDependentColumn('center_of_mass_Y')
time = tableBodyKin.getIndependentColumn()

Now, we can plot the positions.

In [ ]:
import matplotlib.pyplot as plt

# Plot the COM position.
fig, axs = plt.subplots(1, 1, figsize=(7, 5))
axs.plot(time, com_y_pos.to_numpy(), label='COM y-position')
axs.set_xlim([time[0], time[-1]])
axs.set_xlabel('time (s)')
axs.set_ylabel('position (m)')
axs.set_title('Center of Mass Position')
axs.grid()
axs.legend()

plt.show()

Use the following exercises to continuing analyzing the inverse kinematics solution.

**<u>Exercise 1.</u> On your plot of the center of mass position, identify when motion events occur (e.g., deep squat, stand, etc.).**

**<u>Exercise 2.</u> Plot the center of mass velocity in the y-direction. Is it consistent with the position trajectory? Explain.**

## 2.7. Conclusion

In this tutorial you previewed experimental data with the new OpenSim Viewer, performed inverse kinematics to compute joint angle trajectories, and used the `AnalyzeTool` to compute and plot the center of mass of the model during the motion.

## 2.8. Useful Links

> **OpenSim Website:** https://opensim.stanford.edu/
>
> **OpenSim API Documentation:** https://simtk.org/api_docs/opensim/api_docs/
>
> **OpenSim Creator Website:** https://opensimcreator.com/
>
> **SimTK Website:** https://simtk.org/projects/opensim
>
> **Biomechanics of Movement's Course:** https://www.youtube.com/channel/UCDNGy0KKNLQ-ztcL5h2Z6zA
>
> **Webinar on Scaling and Inverse Kinematics in OpenSim:** https://www.youtube.com/watch?v=ZG7wzvQC6eU

## 2.9. Acknowledgments

The experimental marker trajectories of the squat motion were collected by Alberto Casas Ortiz using OpenCap.

Thanks to [OpenSimColab](https://simtk.org/projects/opencolab) project [[9]](https://doi.org/10.1080/10255842.2022.2104607) for creating the first OpenSim Conda package.

## 2.10. References


> [1].   Delp, S. L., Anderson, F. C., Arnold, A. S., Loan, P., Habib, A., John, C. T., Guendelman, E., & Thelen, D. G. (2007). **OpenSim: open-source software to create and analyze dynamic simulations of movement.** *IEEE Transactions on Bio-Medical Engineering*, 54(11), 1940–1950. https://doi.org/10.1109/TBME.2007.901024
>
> [2].   Delp, S. L., Loan, J. P., Hoy, M. G., Zajac, F. E., Topp, E. L., & Rosen, J. M. (1990). **An interactive graphics-based model of the lower extremity to study orthopaedic surgical procedures.** *IEEE Transactions on Bio-Medical Engineering*, 37(8), 757–767. https://doi.org/10.1109/10.102791
>
> [3]. Anderson, F. C., & Pandy, M. G. (1999). **A dynamic optimization solution for vertical jumping in three dimensions.** *Computer Methods in Biomechanics and Biomedical Engineering*, 2(3), 201–231. https://doi.org/10.1080/10255849908907988
>
> [4]. Winter, D. A. (1990). **The biomechanics and motor control of human movement** (2a ed.). *John Wiley & Sons*. https://onlinelibrary.wiley.com/doi/book/10.1002/9780470549148
>
> [5]. Kuo, A. D. (1998). **A least-squares estimation approach to improving the precision of inverse dynamics computations.** *Journal of Biomechanical Engineering*, 120(1), 148–159. https://doi.org/10.1115/1.2834295
>
> [6]. Thelen, D. G., & Anderson, F. C. (2006). **Using computed muscle control to generate forward dynamic simulations of human walking from experimental data.** *Journal of Biomechanics*, 39(6), 1107–1115. https://doi.org/10.1016/j.jbiomech.2005.02.010
>
> [7]. John, C.T., Anderson, F.C., Guendelman, E., Arnold, A.S., Delp, S.L. (21st October 2006). **An algorithm for generating muscle-actuated simulations of long-duration movements.** *Biomedical Computation at Stanford (BCATS) Symposium*, Stanford University, Poster Presentation. https://bcats.stanford.edu/previous_bcats/bcats06/pdf/BCATS_2006_abstract_book.pdf#page=31
>
> [8]. John, C. T., Anderson, F. C., Higginson, J. S., & Delp, S. L. (2013). **Stabilisation of walking by intrinsic muscle properties revealed in a three-dimensional muscle-driven simulation.** *Computer Methods in Biomechanics and Biomedical Engineering*, 16(4), 451–462. https://doi.org/10.1080/10255842.2011.627560
>
> [9] Mokhtarzadeh, H., Jiang, F., Zhao, S., & Malekipour, F. (2022). **OpenColab project: OpenSim in Google colaboratory to explore biomechanics on the web.** *Computer Methods in Biomechanics and Biomedical Engineering*, 1–9. https://doi.org/10.1080/10255842.2022.2104607